---

## 2023-1학기 그래프마이닝 기말고사 프로젝트 
# ✅ COVID-19 가짜 뉴스 탐지

---

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

os.chdir('/content/drive/MyDrive/그래프마이닝/기말프로젝트')

In [3]:
import pandas as pd
import numpy as np

# 1. Data Description
- id : claim id
- label : 0(normal) / 1(fake)
- published_date : 보도 날짜
- keybert_keywords : kebert를 통해 추출한 기사 키워드
- ner_keywords : ner를 통해 추출한 기사 키워드
- youtube0 ~ youtube9 : 해당 키워드로 유트브에 검색했을 때 나오는 상위 10개의 유투브의 제목, 설명 텍스트

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [5]:
# train data: 3868개의 기사로 구성
train.head(1)

,id,label,claim,published_date,keybert_keywords,ner_keywords,youtube0,youtube1,youtube2,youtube3,youtube4,youtube5,youtube6,youtube7,youtube8,youtube9
0,0,0,Did a Vermont Woman Post a Joke About Trump Br...,2018-01-25,"['trump', 'vermont', 'woman']","['vermont', 'trump', 'woman']",Citizen Trump A talk given at the University ...,News Donald Trump has a great woman problem N...,Trump Supporters Remain Despite Vulgar Slur J...,BREAKING Donald trump has a great woman proble...,Revolutionary War SNL The Patriots from New E...,World News Donald Trump has a great woman prob...,Latest News Donald Trump has a great woman pro...,AM Executive Session Gender Matters The U S Pr...,Trump Discusses Tax Bill Gives ;Total Support ...,Chicago s Women s March January This is what ...


In [6]:
# test data: 1856개의 기사로 구성 (test에는 라벨값이 없으므로, 이를 예측하여 submission 해야 함.)
test.head(1)

,id,claim,published_date,keybert_keywords,ner_keywords,youtube0,youtube1,youtube2,youtube3,youtube4,youtube5,youtube6,youtube7,youtube8,youtube9
0,0,According to the CDC so far this year Florida ...,2020-05-27,['pneumonia'],"['florida', 'pneumonia', 'cdc']",Watch Full Coronavirus Coverage May NBC News N...,Watch Full Coronavirus Coverage May NBC News N...,Ask the expert Here ;s why screening is a good...,MORE Questions for an Infectious Disease Speci...,Coronavirus and Cuba,Understanding and Managing Coronavirus in SNF ...,SARS CoV Pandemic Caltech Seminar Day Online S...,CAH Swingbed Initiative with Kerry Dunning Ke...,Diabetics and Novel Coronavirus ▻ Theresa De ...,CDPHE explains how it classifies COVID deaths ...


In [8]:
train.shape, test.shape

((5277, 16), (1856, 15))

In [ ]:
# test 데이터 내 1856개의 노드에 대해 예측된 라벨 (sample에서는 모두 0으로 초기화되어 있음.)
submission.head(1)

,id,label
0,0,0


# 2. Graph

In [12]:
# install dgl
# https://www.dgl.ai/pages/start.html

!pip install -q dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.7/572.7 kB 44.9 MB/s eta 0:0

In [13]:
import numpy as np
from scipy.sparse import coo_matrix
import torch
import dgl

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


## 2-1) adjacency matrix 구축

In [ ]:
# adjacency matrix 구축 예시: 같은 월에 작성된 claim끼리 연결
#train['published_date'] = pd.to_datetime(train['published_date'], format='%Y-%m-%d', errors='raise') 
#train.head(1)

In [ ]:
# adjacency matrix 초기화
#dic_array = np.zeros((len(train),len(train)))
#print(dic_array.shape)
#print(dic_array.sum())

In [14]:
# adjacency matrix 구축: 

# 해당 열 list로 변환
train['keybert_keywords'] = train['keybert_keywords'].apply(eval)
train['ner_keywords'] = train['ner_keywords'].apply(eval)
test['keybert_keywords'] = test['keybert_keywords'].apply(eval)
test['ner_keywords'] = test['ner_keywords'].apply(eval)

# adjacency matrix 초기화
dic_array = np.zeros((len(train),len(train)))
dic_array_test = np.zeros((len(test),len(test)))
print(dic_array.shape, dic_array.sum())
print(dic_array_test.shape, dic_array_test.sum())

(5277, 5277) 0.0
(1856, 1856) 0.0


In [15]:
# train
src=[]
tg=[]

for i, (i_claim, i_m) in enumerate(zip(train.claim, train.ner_keywords)):
  for j, (j_claim, j_m) in enumerate(zip(train.claim, train.ner_keywords)):

    weight = len(set(i_m).intersection(j_m))
    if weight > 1 :
      dic_array[i][j] = weight #
      src.append(i)
      tg.append(j)


print(dic_array.shape)
print(dic_array.sum())

(5277, 5277)
208749.0


In [16]:
# train edge
df = {
    'src':src,
    'tg':tg,
    'weight':weight
}
train_edge = pd.DataFrame(data=df, columns = ['src','tg','weight'])
train_edge

,src,tg,weight
0,0,0,3
1,0,1495,3
2,0,3326,3
3,1,1,3
4,2,2,3
...,...,...,...
100752,5275,5206,3
100753,5275,5246,3
100754,5275,5248,3
100755,5275,5275,3


In [17]:
src_test=[]
tg_test=[]


for i, (i_claim, i_m) in enumerate(zip(test.claim, train.ner_keywords)):
  for j, (j_claim, j_m) in enumerate(zip(test.claim, train.ner_keywords)):

    weight_test = len(set(i_m).intersection(j_m))
    if weight_test > 1 :
      dic_array_test[i][j] = weight #
      src_test.append(i)
      tg_test.append(j)

print(dic_array_test.shape, dic_array_test.sum())

(1856, 1856) 38451.0


In [18]:
df_test = {
    'src':src_test,
    'tg':tg_test,
    'weight':weight_test
}
test_edge = pd.DataFrame(data=df_test, columns = ['src','tg','weight'])
test_edge

,src,tg,weight
0,0,0,3
1,0,1495,3
2,1,1,3
3,2,2,3
4,2,175,3
...,...,...,...
12812,1854,1595,3
12813,1854,1636,3
12814,1854,1771,3
12815,1854,1854,3


In [19]:
# 완성된 Adjacency matrix 및 엣지 합 (개수X, 합O)
print(dic_array.shape, dic_array.sum())
print(dic_array_test.shape, dic_array_test.sum())

(5277, 5277) 208749.0
(1856, 1856) 38451.0


In [20]:
dic_array

array([[3., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 3., 0., 0.],
       [0., 0., 0., ..., 0., 3., 0.],
       [0., 0., 0., ..., 0., 0., 3.]])

In [21]:
dic_array_test

array([[3., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.],
       [0., 0., 3., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 3., 0., 0.],
       [0., 0., 0., ..., 0., 3., 0.],
       [0., 0., 0., ..., 0., 0., 3.]])

In [22]:
dic_array.shape

(5277, 5277)

## 2-2) feature

In [23]:
train_corpus = train['claim'].tolist()
test_corpus = test['claim'].tolist()

len(train_corpus) , len(test_corpus)

(5277, 1856)

In [24]:
! pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=15057fa02ae9651781474afad9d61caaad11d1444e17355bf03e749935320a35
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [25]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer('bert-base-nli-mean-tokens')
train_corpus_embeddings = embedder.encode(train_corpus)
train_corpus_embeddings.shape, train_corpus_embeddings

test_corpus_embeddings = embedder.encode(test_corpus)
test_corpus_embeddings.shape, test_corpus_embeddings

((1856, 768),
 array([[-0.6574448 ,  0.00423064, -0.21266967, ..., -0.13565542,
          1.1537006 ,  0.22186403],
        [ 0.10222992, -0.00407351,  1.2712792 , ...,  0.05499515,
         -0.2581544 ,  0.41784534],
        [ 0.37054715,  0.6438513 ,  1.0204334 , ..., -0.0615061 ,
         -0.27258477,  0.5787992 ],
        ...,
        [-0.0251382 , -0.16165885,  0.01104116, ..., -1.0120296 ,
          0.57121915,  0.44252768],
        [ 0.07177757,  0.7794703 ,  0.06981117, ...,  0.02873591,
         -0.47539532,  0.7061203 ],
        [-0.19671902,  0.9214573 , -0.08972804, ...,  0.42101714,
          0.4186466 , -0.05303368]], dtype=float32))

In [26]:
train['embedding'] = train_corpus_embeddings.tolist()
test['embedding'] = test_corpus_embeddings.tolist()

## 2-3) build graph

In [28]:
node_feat = torch.from_numpy(train_corpus_embeddings)
labels = train['label'].values  
edge_feat = torch.from_numpy(train_edge['weight'].to_numpy())
edges_src = torch.from_numpy(train_edge['src'].to_numpy())
edges_tg = torch.from_numpy(train_edge['tg'].to_numpy())

g_train = dgl.graph((edges_src, edges_tg), num_nodes=len(train)) 

g_train.ndata['feat'] = node_feat
g_train.edata['weight'] = edge_feat
g_train = dgl.add_self_loop(g_train)

In [29]:
g_train

Graph(num_nodes=5277, num_edges=106034,
      ndata_schemes={'feat': Scheme(shape=(768,), dtype=torch.float32)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.int64)})

In [48]:
## train indices

import numpy as np

train_percentage = 0.8
num_train = int(len(train) * train_percentage)

# Random
train_indices = np.random.choice(len(train), size=num_train, replace=False)
train_indices 

array([3732, 4021, 2535, ..., 1189, 2877,  825])

In [49]:
train_mask = np.zeros(len(train), dtype=bool)  # Initialize
train_mask[train_indices] = True  
valid_mask = np.logical_not(train_mask)  

In [50]:
g_train.ndata['label'] = torch.tensor(labels)  # Assign labels to the 'label' node feature in the graph
g_train.ndata['train_mask'] = torch.tensor(train_mask)  # Assign train_mask to the 'train_mask' node feature in the graph
g_train.ndata['valid_mask'] = torch.tensor(valid_mask)  # Assign valid_mask to the 'valid_mask' node feature in the graph

In [51]:
node_feat = torch.from_numpy(test_corpus_embeddings)
edge_feat = torch.from_numpy(test_edge['weight'].to_numpy())
edges_src = torch.from_numpy(test_edge['src'].to_numpy())
edges_tg = torch.from_numpy(test_edge['tg'].to_numpy())

g_test = dgl.graph((edges_src, edges_tg), num_nodes=len(test)) 

g_test.ndata['feat'] = node_feat
g_test.edata['weight'] = edge_feat
g_test = dgl.add_self_loop(g_test)

test_mask = np.zeros(len(test), dtype=bool)  # Initialize 
g_test.ndata['test_mask'] = torch.tensor(test_mask)
g_test

Graph(num_nodes=1856, num_edges=14673,
      ndata_schemes={'feat': Scheme(shape=(768,), dtype=torch.float32), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.int64)})

In [52]:
# graph save
dgl.save_graphs('graph_train.bin', [g_train])

dgl.save_graphs('graph_test.bin', [g_test])

In [53]:
loaded_graphs, _ = dgl.load_graphs('graph_test.bin')
loaded_graph = loaded_graphs[0]  

In [54]:
loaded_graphs, _ = dgl.load_graphs('graph_train.bin')
loaded_graph = loaded_graphs[0]  

In [55]:
train_g = loaded_graph.subgraph(train_mask.nonzero()[0])
valid_g = loaded_graph.subgraph(valid_mask.nonzero()[0])

In [56]:
train_mask, valid_mask

(array([ True,  True, False, ..., False, False,  True]),
 array([False, False,  True, ...,  True,  True, False]))

In [57]:
train_g

Graph(num_nodes=4221, num_edges=69602,
      ndata_schemes={'valid_mask': Scheme(shape=(), dtype=torch.uint8), 'train_mask': Scheme(shape=(), dtype=torch.uint8), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(768,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [58]:
valid_g 

Graph(num_nodes=1056, num_edges=6146,
      ndata_schemes={'valid_mask': Scheme(shape=(), dtype=torch.uint8), 'train_mask': Scheme(shape=(), dtype=torch.uint8), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(768,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'weight': Scheme(shape=(), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)})

In [59]:
train_g.ndata['feat']

tensor([[-0.5393,  0.0806, -0.4042,  ..., -0.0734, -0.5007,  0.5461],
        [ 0.2018, -0.3245, -0.7264,  ...,  1.2555, -0.0048, -0.4929],
        [ 0.7045,  0.1956,  0.0151,  ..., -0.3604, -0.8487,  0.5905],
        ...,
        [-0.0037,  0.2216, -0.9410,  ..., -1.1994,  0.5010,  0.3341],
        [ 0.4145,  0.9162,  0.7886,  ...,  0.1861,  0.1406,  0.0321],
        [ 0.3218,  0.5982, -0.6228,  ...,  0.0424,  0.6852, -0.4528]])

# 3) Model

In [61]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.conv1 = dgl.nn.GraphConv(in_feats, hidden_size)
        self.conv2 = dgl.nn.GraphConv(hidden_size, num_classes)
    
    def forward(self, g, features):
        x = F.relu(self.conv1(g, features))
        x = self.conv2(g, x)
        return x

in_feats = train_g.ndata['feat'].shape[1]  
hidden_size = 64  
num_classes = 2
model = GCN(in_feats, hidden_size, num_classes)

In [63]:
# the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop
def train(model, graph, optimizer, criterion):
    model.train()  
    optimizer.zero_grad()  

    features = graph.ndata['feat']
    labels = graph.ndata['label']
    train_mask = graph.ndata['train_mask']

    # Forward pass
    logits = model(graph, features)
    loss = criterion(logits[train_mask], labels[train_mask])

    # Backward pass
    loss.backward()
    optimizer.step()

    return loss.item()

In [64]:
# Validation 
def compute_accuracy(model, graph):
    model.eval() 

    features = graph.ndata['feat']
    labels = graph.ndata['label']
    valid_mask = graph.ndata['valid_mask']

    # Forward pass
    with torch.no_grad():
        logits = model(graph, features)
        _, predicted = torch.max(logits[valid_mask], 1)
        correct = (predicted == labels[valid_mask]).sum().item()
        accuracy = correct / valid_mask.sum().item()

    return accuracy

In [65]:
# Train the model
num_epochs = 50

for epoch in range(num_epochs):
    loss = train(model, train_g, optimizer, criterion)
    accuracy = compute_accuracy(model, valid_g)
    if epoch % 10 == 0:
      print(f'Epoch: {epoch+1}, Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')

/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:352: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), "Cannot convert view " \
<ipython-input-63-ba4608033e1c>:16: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  loss = criterion(logits[train_mask], labels[train_mask])
/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:200: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at ../aten/src/ATen/native/IndexingUtils.h:27.)
  Variable._execution_engine.run_backward(  # Calls 

Epoch: 1, Loss: 0.7603, Validation Accuracy: 0.6581
Epoch: 11, Loss: 0.6069, Validation Accuracy: 0.6184
Epoch: 21, Loss: 0.5892, Validation Accuracy: 0.6714
Epoch: 31, Loss: 0.5600, Validation Accuracy: 0.6951
Epoch: 41, Loss: 0.5329, Validation Accuracy: 0.6989


In [66]:
# evaluation

model.eval()  
with torch.no_grad():
    logits = model(g_test, g_test.ndata['feat'])
    predicted_labels = logits.argmax(dim=1)

print(len(predicted_labels))

1856


# 결과 확인

In [67]:
submission['label'] = predicted_labels
submission

,id,label
0,0,0
1,1,0
2,2,0
3,3,1
4,4,1
...,...,...
1851,1851,0
1852,1852,1
1853,1853,1
1854,1854,1


In [68]:
submission['label'].value_counts()

0    1029
1     827
Name: label, dtype: int64

In [69]:
submission.to_csv('0527_epoch50_submission.csv', index=False) # 0.40569